In [238]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [220]:
TARGET = 'trip_duration'

In [242]:
df = pd.read_csv("../input/train/train.csv",index_col="id")

#Filtre : on prend uniquement les trajets qui dure moins d'une heure, et les trajets avec des passagers 
df = df[(df['trip_duration']<3600)]
df = df[(df['passenger_count']>0)]
df.shape

(1427007, 10)

In [224]:
def create_dataset(df, features=None, target=TARGET):
    X = df[features]
    y = df[target]
    return X, y

**Creation du dataset**

In [225]:
X , y = create_dataset(df,features=df.columns)
X= X.drop(['trip_duration','store_and_fwd_flag','vendor_id','dropoff_datetime'],axis=1)

((1446251, 10), (1446251,))

In [227]:
df_test = pd.read_csv('../input/test/test.csv',index_col='id')
df_test=df_test.drop(['store_and_fwd_flag','vendor_id'],axis=1)

,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
id,,,,,,
id3004672,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680
id3505355,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403
id1217141,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523
id2150126,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469
id1598245,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890


In [228]:
def split_date(df,name_column):
        date=pd.to_datetime(df[name_column])
        df[name_column+'_second']=date.dt.second
        df[name_column+'_minute']=date.dt.minute
        df[name_column+'_hour']=date.dt.hour
        df[name_column+'_year']=date.dt.year
        df[name_column+'_day']=date.dt.day
        df= df.drop([name_column],axis=1)
        return df
        

In [229]:
X = split_date(X,'pickup_datetime')
df_test = split_date(df_test,'pickup_datetime')

**Cross Validation**

In [231]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()


In [232]:
from sklearn.model_selection import (
    cross_val_score, ShuffleSplit)
from sklearn.metrics import accuracy_score

In [233]:
rs = ShuffleSplit(n_splits=3, test_size=.25, train_size=.12)
cv_scores = -cross_val_score(rf, X, y, cv=rs)
cv_scores.mean()

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-0.7209074640497853

In [234]:
rf.fit(X,y)

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [235]:
df_test.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_datetime_second,pickup_datetime_minute,pickup_datetime_hour,pickup_datetime_year,pickup_datetime_day
id,,,,,,,,,,
id3004672,1,-73.988129,40.732029,-73.990173,40.756680,58,59,23,2016,30
id3505355,1,-73.964203,40.679993,-73.959808,40.655403,53,59,23,2016,30
id1217141,1,-73.997437,40.737583,-73.986160,40.729523,47,59,23,2016,30
id2150126,1,-73.956070,40.771900,-73.986427,40.730469,41,59,23,2016,30
id1598245,1,-73.970215,40.761475,-73.961510,40.755890,33,59,23,2016,30


In [236]:
y_pred= rf.predict(df_test)
y_pred.mean()

833.3760476953739

In [237]:
submission = pd.read_csv('../input/sample_submission/sample_submission.csv')
submission['trip_duration']=y_pred
submission.to_csv('submission.csv',index=False)